# Deepfake Detection Model Training
This notebook trains a deep learning model to classify real vs fake (AI-generated) faces using Transfer Learning with EfficientNetB4.

## Model Architecture:
- **Base Model**: EfficientNetB4 (pre-trained on ImageNet)
- **Custom Head**: Global Average Pooling → Dense Layers → Dropout → Sigmoid Output
- **Loss Function**: Binary Crossentropy
- **Optimizer**: Adam with learning rate scheduling

## Dataset Structure:
```
dataset/
├── train/
│   ├── real/
│   └── fake/
└── validation/
    ├── real/
    └── fake/
```

In [ ]:
# Import Required Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB4, ResNet50, Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Configuration and Hyperparameters
IMG_SIZE = 224  # EfficientNetB4 input size
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0001
FINE_TUNE_EPOCHS = 20
FINE_TUNE_AT = 100  # Layer to start fine-tuning from

# Paths
TRAIN_DIR = '../dataset/train'
VAL_DIR = '../dataset/validation'
MODEL_SAVE_PATH = './saved_models/deepfake_detector_efficientnet.h5'
WEIGHTS_SAVE_PATH = './saved_models/deepfake_detector_weights.h5'

# Create model directory
os.makedirs('./saved_models', exist_ok=True)
os.makedirs('./logs', exist_ok=True)

In [ ]:
# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Only rescaling for validation
val_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True,
    seed=42
)

# Load validation data
val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print(f"\nClass Indices: {train_generator.class_indices}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")

In [ ]:
# Visualize sample images
def plot_sample_images(generator, num_images=8):
    """Plot sample images from the dataset"""
    plt.figure(figsize=(15, 8))
    
    # Get a batch of images
    images, labels = next(generator)
    
    for i in range(min(num_images, len(images))):
        plt.subplot(2, 4, i + 1)
        plt.imshow(images[i])
        label = 'FAKE' if labels[i] == 0 else 'REAL'
        color = 'red' if labels[i] == 0 else 'green'
        plt.title(f'Label: {label}', color=color, fontweight='bold')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

plot_sample_images(train_generator)

In [ ]:
# Build the Model using Transfer Learning
def create_model(base_model_name='EfficientNetB4'):
    """Create deepfake detection model with transfer learning"""
    
    # Load pre-trained base model
    if base_model_name == 'EfficientNetB4':
        base_model = EfficientNetB4(
            include_top=False,
            weights='imagenet',
            input_shape=(IMG_SIZE, IMG_SIZE, 3)
        )
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(
            include_top=False,
            weights='imagenet',
            input_shape=(IMG_SIZE, IMG_SIZE, 3)
        )
    elif base_model_name == 'Xception':
        base_model = Xception(
            include_top=False,
            weights='imagenet',
            input_shape=(IMG_SIZE, IMG_SIZE, 3)
        )
    
    # Freeze base model layers initially
    base_model.trainable = False
    
    # Build the model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    return model, base_model

# Create the model
model, base_model = create_model('EfficientNetB4')

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

model.summary()

In [ ]:
# Define Callbacks
checkpoint = ModelCheckpoint(
    MODEL_SAVE_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

tensorboard = TensorBoard(
    log_dir='./logs',
    histogram_freq=1
)

callbacks = [checkpoint, early_stop, reduce_lr, tensorboard]

In [ ]:
# Train the Model (Initial Training with Frozen Base)
print("\n" + "="*60)
print("Starting Initial Training (Frozen Base Model)")
print("="*60 + "\n")

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Fine-Tuning: Unfreeze some layers of the base model
print("\n" + "="*60)
print("Starting Fine-Tuning")
print("="*60 + "\n")

# Unfreeze the base model
base_model.trainable = True

# Freeze layers before FINE_TUNE_AT
for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE / 10),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

# Continue training
history_fine = model.fit(
    train_generator,
    epochs=EPOCHS + FINE_TUNE_EPOCHS,
    initial_epoch=len(history.history['loss']),
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Plot Training History
def plot_training_history(history, history_fine=None):
    """Plot training and validation metrics"""
    
    # Combine histories if fine-tuning was done
    if history_fine:
        acc = history.history['accuracy'] + history_fine.history['accuracy']
        val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
        loss = history.history['loss'] + history_fine.history['loss']
        val_loss = history.history['val_loss'] + history_fine.history['val_loss']
    else:
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
    
    epochs_range = range(len(acc))
    
    plt.figure(figsize=(16, 6))
    
    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy', linewidth=2)
    plt.plot(epochs_range, val_acc, label='Validation Accuracy', linewidth=2)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss', linewidth=2)
    plt.plot(epochs_range, val_loss, label='Validation Loss', linewidth=2)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.title('Training and Validation Loss', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('./logs/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_training_history(history, history_fine)

In [ ]:
# Evaluate the Model
print("\n" + "="*60)
print("Model Evaluation on Validation Set")
print("="*60 + "\n")

# Load best model
best_model = keras.models.load_model(MODEL_SAVE_PATH)

# Evaluate
results = best_model.evaluate(val_generator, verbose=1)
print(f"\nValidation Loss: {results[0]:.4f}")
print(f"Validation Accuracy: {results[1]:.4f}")
print(f"Validation Precision: {results[2]:.4f}")
print(f"Validation Recall: {results[3]:.4f}")
print(f"Validation AUC: {results[4]:.4f}")

In [ ]:
# Generate Predictions and Confusion Matrix
# Reset generator
val_generator.reset()

# Get predictions
predictions = best_model.predict(val_generator, verbose=1)
y_pred = (predictions > 0.5).astype(int).flatten()
y_true = val_generator.classes

# Classification Report
print("\n" + "="*60)
print("Classification Report")
print("="*60 + "\n")
print(classification_report(y_true, y_pred, target_names=['FAKE', 'REAL']))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['FAKE', 'REAL'], yticklabels=['FAKE', 'REAL'])
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.savefig('./logs/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ROC Curve
fpr, tpr, thresholds = roc_curve(y_true, predictions)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.savefig('./logs/roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Save Model in Different Formats
print("\n" + "="*60)
print("Saving Model")
print("="*60 + "\n")

# Save as .h5
best_model.save(MODEL_SAVE_PATH)
print(f"✅ Model saved as: {MODEL_SAVE_PATH}")

# Save weights only
best_model.save_weights(WEIGHTS_SAVE_PATH)
print(f"✅ Weights saved as: {WEIGHTS_SAVE_PATH}")

# Save in TensorFlow SavedModel format (for production)
tf_save_path = './saved_models/deepfake_detector_savedmodel'
best_model.save(tf_save_path, save_format='tf')
print(f"✅ TensorFlow SavedModel saved at: {tf_save_path}")

# Save model architecture as JSON
model_json = best_model.to_json()
with open('./saved_models/model_architecture.json', 'w') as json_file:
    json_file.write(model_json)
print(f"✅ Model architecture saved as JSON")

print("\n🎉 Model training completed successfully!")

In [ ]:
# Test Prediction Function
def predict_image(image_path, model):
    """Predict if an image is real or fake"""
    from tensorflow.keras.preprocessing import image
    
    # Load and preprocess image
    img = image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    
    # Predict
    prediction = model.predict(img_array, verbose=0)[0][0]
    
    # Interpret result
    if prediction < 0.5:
        result = "FAKE"
        confidence = (1 - prediction) * 100
    else:
        result = "REAL"
        confidence = prediction * 100
    
    return result, confidence, prediction

# Example usage (uncomment and provide test image path)
# test_image_path = '../test_images/sample.jpg'
# result, confidence, score = predict_image(test_image_path, best_model)
# print(f"Result: {result}")
# print(f"Confidence: {confidence:.2f}%")
# print(f"Raw Score: {score:.4f}")

## Model Training Complete! 🎉

### Next Steps:
1. Test the model with sample images
2. Integrate with Flask/FastAPI backend
3. Deploy to production

### Model Files Generated:
- `deepfake_detector_efficientnet.h5` - Complete model
- `deepfake_detector_weights.h5` - Model weights only
- `deepfake_detector_savedmodel/` - TensorFlow SavedModel format
- `model_architecture.json` - Model architecture

### Performance Metrics:
Check the logs folder for:
- Training history plots
- Confusion matrix
- ROC curve
- TensorBoard logs

In [ ]:
# Alternative: Using Pretrained Model from Kaggle
# If you don't have time to train, download a pretrained model

import os
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Option 1: Download from Kaggle (Real and Fake Face Detection dataset)
# Dataset: https://www.kaggle.com/datasets/ciplab/real-and-fake-face-detection
# This dataset contains real and fake face images perfect for deepfake detection

def download_pretrained_model_from_kaggle():
    """
    Instructions for downloading and using pretrained model from Kaggle
    """
    print("To use a pretrained model instead of training:")
    print("1. Go to: https://www.kaggle.com/datasets/ciplab/real-and-fake-face-detection")
    print("2. Download the dataset (contains real and fake face images)")
    print("3. Extract to '../dataset/' folder")
    print("4. Use the dataset structure:")
    print("   dataset/")
    print("   ├── train/")
    print("   │   ├── real/")
    print("   │   └── fake/")
    print("   └── validation/")
    print("       ├── real/")
    print("       └── fake/")
    print()
    print("5. If you find a pretrained model (.h5 file) on Kaggle,")
    print("   download it and use the code below to load it:")

    # Example code to load pretrained model
    pretrained_model_path = "./pretrained_models/deepfake_detector.h5"

    if os.path.exists(pretrained_model_path):
        try:
            model = keras.models.load_model(pretrained_model_path)
            print(f"✅ Pretrained model loaded from {pretrained_model_path}")

            # Test the model
            print("Model summary:")
            model.summary()

            return model
        except Exception as e:
            print(f"❌ Error loading pretrained model: {e}")
            return None
    else:
        print(f"❌ Pretrained model not found at {pretrained_model_path}")
        print("Please download a pretrained model from Kaggle and place it there.")
        return None

# Uncomment to try loading pretrained model
# pretrained_model = download_pretrained_model_from_kaggle()

# If you have a pretrained model, you can skip training and go directly to evaluation
# Just replace 'model' with 'pretrained_model' in the evaluation cells above

print("\n" + "="*60)
print("PRETRAINED MODEL INSTRUCTIONS")
print("="*60)
print("If you want to skip training entirely:")
print("1. Find a deepfake detection model on Kaggle")
print("2. Download the .h5 model file")
print("3. Place it in './pretrained_models/' folder")
print("4. Uncomment and run the download_pretrained_model_from_kaggle() function")
print("5. Use the loaded model for inference instead of training")
print("="*60)